# Bayesian Updating in Golf
## Teddy Murphey
## Dr. Michael Klucznik, Dr. Chris Bopp, Dr. Chris Hill

### Developing Bayesian Prior Distributions
The first step in creating a bayesian updating model for golf prediction is the development of a prior distribution. We accomplish this by computing average distributions from historically unbiased tournaments. We select the 2022 CJ Cup as its adjusted score relative to par was just +0.05, a significantly low differential. The advantage of using an unbiased tournament is that golfers performed exceptionally close to "normal." Golf course metrics such as slope rating and course rating (which we will use) are based on "normal" scoring.

In [ ]:
# import scoring average from local csv

In [ ]:
# Check working directory
import os
print(os.getcwd())

# Import csv and check its properties
import pandas as pd
import numpy as np
df = pd.read_csv('CJ_Cup_2022.csv')
print(df.shape, df.columns)

Here, we find the probability of each golf score by dividing each distinct score count by the total.

In [ ]:
# Count total 
totalResults = df['Eagles'].sum() + df['Birdies'].sum() + df['Pars'].sum() + df['Bogeys'].sum() + df['Dbls+'].sum()
print(totalResults)

# Compute probability for each score
eagleProb = df['Eagles'].sum() / totalResults
birdieProb = df['Birdies'].sum() / totalResults
parProb = df['Pars'].sum() / totalResults
bogeyProb = df['Bogeys'].sum() / totalResults
doublePlusProb = df['Dbls+'].sum() / totalResults

# Here we see the probability of each score
# Note that this includes all possible scores on a golf hole except a hole-in-one on a par 4/5 and an albatross (2 on a par 5)
print("Score Probabilities (in %):\nEagle:", eagleProb * 100, "\nBirdie:", birdieProb * 100, "\nPar:", parProb * 100, "\nBogey:", bogeyProb * 100, "\nDouble Bogey Plus:", doublePlusProb * 100)
print("Check that total probability adds to 1:", eagleProb + birdieProb + parProb + bogeyProb + doublePlusProb)

### Required Inputs for Analysis
To predict match play results, we need to know a few things about our golfers and the course.
1. Golfers' handicaps
    - Score distributions are adjusted based on each handicap
    - This is the metric that will change based on hole results
2. Course rating
    - No two courses are the same
    - The official rating helps us adjust for course difficulty
3. Slope rating
    - Difficult courses hurt bad golfers **more** than good golfers
    - This rating accounts for the difference between how a bogey golfer and a scratch golfer should perform
4. We need these additional values for **each** hole:
    1. Hole handicap index: a number between 1 and 18
    2. Each golfer's score on the hole: birdie, par, etc.

### Collecting Global Variables from User
The user will input the necessary variables for our analysis here. Players' **initial handicap indices**, the course's **total par**, the course's **rating**, and the **slope rating** will not change.

In [ ]:
# Start by requesting user for all relevant inputs
# These first six inputs will be used for the entire match; they do not change each hole

# Note that the USGA only provides statistics for two genders: male and female
genderA = input("Gender associated with Player A's USGA membership (M/F): ")
genderB = input("Gender associated with Player B's USGA membership (M/F): ")

initialHandicapA = float(input("Enter Player A's handicap: "))
initialHandicapB = float(input("Enter Player B's handicap: "))

coursePar = int(input("Course's total par: "))
courseRating = float(input("Enter course rating: "))
slopeRating = float(input("Enter slope rating: "))

### Calculated Variables
Using inputted variables, we then calculate some more useful variables. To compare players in 1v1 matchplay, their handicaps are subtracted. But we can't simply subtract the players' handicap indices, since each of their playing handicaps differ depending on the course. We need to find a **course handicap** for each player by accounting for the **slope rating**. Note that we collected **course rating** from the user, but will not use it. This is because the current USGA standards, which we will use, require only **slope rating**. **Course rating** is, however, used by other organizations when calculating **course handicap**, so it may come in handy in certain future situations.

In [ ]:
# These calculations use the USGA's formula for course handicap; C.H. = (H.I. * S.R.) / 113
courseHandicapA = (initialHandicapA * slopeRating) / 113
courseHandicapB = (initialHandicapB * slopeRating) / 113
print("Player A's course handicap:", courseHandicapA)
print("Player B's course handicap:", courseHandicapB)

After finding **course handicaps** for each player, we subtract the smaller from the larger. Handicaps, in general, determine how many extra strokes players are allotted per round. Simply put, a player with a **course handicap** of 6 will have a stroke subtracted from their score on each of the course's 6 hardest holes.

To directly compare two players, we take the difference in their handicaps and give that many strokes to the worse player. So, rather than a 16 handicap getting strokes on the 16 hardest holes and a 12 handicap getting strokes on the 12 hardest holes, we simply give the former player 16 - 12 = 4 strokes. The first player will get a stroke subtracted on each of the 4 hardest holes, which is much easier to keep track of, but is effectively the same handicap (for match purposes).

In [ ]:
# Beyond subtraction, we round off to the nearest whole number since golf scores, and, thus, handicap-adjusted scores, must be whole.
playingDifferential = round(abs(courseHandicapA - courseHandicapB))
print('Strokes to be given to higher handicapper: ', playingDifferential)

### Match Procedure
In my proposed live handicapping system, we will use bayesian updating to make live adjustments to each players' handicaps. Using our prior score distribution, we will calculate the posterior probability of certain scores being achieved. These will be compared with the players' reported scores. If a reported score is determined to be statistically improbable, then a handicap adjustment will be performed. Incorrect handicaps are a prominent problem in golf, whether intentional ("a sandbagger") or unintentional. Making adjustments based on impossible outcomes is how we will try to fix the problem.

Procedure for each hole (essentially pseudocode for a function):
1. Collect necessary inputs from user
    1. Hole par: 3-5
    1. Hole handicap index: 1-18
    2. Player A score: birdie, par, etc.
    3. Player B score: birdie, par, etc.
2. Compute probabilities for each player's score given their handicap.
    1. P(Score|Handicap) = prior distribution (adjusted for handicap; e.g., if getting a stroke, par is 20.139% chance, not birdie)
    2. P(Handicap) = ???
    3. P(Score) = ???
    4. P(Handicap|Score) = P(Score|Handicap) * (P(Handicap) / P(Score))

Some variables change hole-to-hole, so we will collect these each hole.

In [ ]:
# Hole handicaps indicate the difficulty of the hole; 1 is hardest, 18 is easiest
# If a player's stroke allowance is >= hole handicap, then the player receives a stroke
# In the case where a player's handicap is below 0 (indicated by a +), they give a stroke on the easier holes
# (a +3 handicap has a stroke added on the holes handicapped 18, 17, and 16)
# In the case where a player's handicap is above 18, they receive multiple strokes on the hardest holes
# (a 22 handicap receives a stroke on the holes handicapped 1-18 and an additional, second stroke on the holes handicapped 1-4)
holePar = float(input("Enter the current hole's par (3-5): "))
holeIndex = float(input("Enter the current hole's handicap index: "))
scoreA = float(input("Enter Player A's numeric score: "))
scoreB = float(input("Enter Player B's numeric score: "))


After the players report the par, index, and scores, we need to calculate the score type. We have our prior data in terms of birdie, par, etc., so we need scores converted to this. We write a function to handle this since it may need repeated. We then use our new function to convert each player's score to the categorical equivalent. Keep in mind, since we are evaluating the validity of players' handicaps, we want their raw score to analyze. So, we do not actually grant the allotted strokes here.

In [ ]:
def convertScore(score, par):
    """
    Converts a player's numeric score to string
    
    Parameters:
        score (int): strokes taken by player on hole
        par (int): par for the hole (usually 3, 4, or 5)
        
    Returns:
        string: name for the hole result (Eagle, Birdie, Par, Bogey, Double Bogey+)
    
    """
    
    # Dictionary to associate number with categorical score
    d1 = {-2 : "Eagle",
         -1 : "Birdie",
          0 : "Par",
          1 : "Bogey"}
    
    relativeToPar = score - par
    
    if relativeToPar < -2:
        return "None"
        
    elif relativeToPar > 1:
        return "Double Bogey+"
    
    else:
        return d1[relativeToPar]

In [ ]:
catScoreA = convertScore(scoreA, holePar)
catScoreB = convertScore(scoreB, holePar)

print("Player A scored a", catScoreA, "and Player B scored a", catScoreB)

Now that we have a score from each player, we find the probability of this score by adjusting for their handicap.

In [ ]:
if (courseHandicapA - holeIndex) >= 36:
    adjCatScoreA = convertScore(scoreA - 3, holePar)
elif (courseHandicapA - holeIndex) >= 18:
    adjCatScoreA = convertScore(scoreA - 2, holePar)
elif (courseHandicapA - holeIndex) >= 0:
    adjCatScoreA = convertScore(scoreA - 1, holePar)
else:
    adjCatScoreA = catScoreA
    
if (courseHandicapB - holeIndex) >= 36:
    adjCatScoreB = convertScore(scoreB - 3, holePar)
elif (courseHandicapB - holeIndex) >= 18:
    adjCatScoreB = convertScore(scoreB - 2, holePar)
elif (courseHandicapB - holeIndex) >= 0:
    adjCatScoreB = convertScore(scoreB - 1, holePar)
else:
    adjCatScoreB = catScoreB
    
print("Adjusting for handicap, Player A scored a net", adjCatScoreA)
print("Adjusting for handicap, Player B scored a net", adjCatScoreB)

Now we calculate the probability of that score.

In [ ]:
# Dictionary to associate categorical score with probability
d2 = {"Eagle" : eagleProb,
     "Birdie" : birdieProb,
     "Par" : parProb,
     "Bogey" : bogeyProb,
     "Double Bogey+" : doublePlusProb,
     "None" : 0}

probPlayerA = d2[adjCatScoreA]
probPlayerB = d2[adjCatScoreB]

print("Player A's result of a", catScoreA, "( net", adjCatScoreA, ") has a probability of", probPlayerA)
print("Player B's result of a", catScoreB, "( net", adjCatScoreB, ") has a probability of", probPlayerB)

Now that we have the conditional probability of the players' score given their handicap, **probPlayerA** and **probPlayerB**, we need to find the probability of that handicap. Here, we need to find a distribution of handicaps. Here, we again rely on the USGA and [their latest handicapping statistics](https://www.usga.org/handicapping/us-stats-static.html). These probabilities are current as of January 4th, 2024.

Note that the USGA only provides statistics for two genders: male and female.

In [ ]:
# Import handicapping distributions from local csv
hcProbMale = pd.read_csv('Handicap_Probabilities_Male.csv')
hcProbFemale = pd.read_csv('Handicap_Probabilities_Female.csv')

print("Male handicap distribution data:\n", hcProbMale.shape, "\n", hcProbMale)
print("Female handicap distribution data:\n", hcProbFemale.shape, "\n", hcProbFemale)

# We then attach the proper distribution to each player based on playing gender
if genderA == 'M':
    hcProbA = hcProbMale
else:
    hcProbA = hcProbFemale
    
if genderB == 'M':
    hcProbB = hcProbMale
else:
    hcProbB = hcProbFemale

Next, we convert each player's handicap to its associated range, as expressed in the above distributions. This will allow us to lookup each handicap using the table.

In [ ]:
def convertHandicapToRange(handicap):
    """
    Bins a numeric handicap in its associated range as expressed in USGA table
    
    Parameter:
        handicap (float): numeric handicap index, ranges from negative single digits (no exact minimum) to a hard maximum of 54
    
    Returns:
        int: corresponding row in the DataFrame
    """
    
    if handicap < 0:
        return 0
    elif handicap < 5:
        return 1
    elif handicap < 10:
        return 2
    elif handicap < 15:
        return 3
    elif handicap < 20:
        return 4
    elif handicap < 25:
        return 5
    elif handicap < 30:
        return 6
    elif handicap < 35:
        return 7
    elif handicap < 40:
        return 8
    elif handicap < 45:
        return 9
    elif handicap < 50:
        return 10
    elif handicap <= 54:
        return 11
    else:
        return "Invalid handicap"

We can now find the prior probability of each players' handicap. It won't take as much evidence for us to discount a player with an already-rare handicap.

In [ ]:
hcProbBinA = hcProbA.iloc[convertHandicapToRange(courseHandicapA)]["Percentage of Golfers"]
hcProbBinB = hcProbB.iloc[convertHandicapToRange(courseHandicapB)]["Percentage of Golfers"]

print("Player A's initial handicap of", initialHandicapA, "has a prior probability of", hcProbBinA)
print("Player B's initial handicap of", initialHandicapB, "has a prior probability of", hcProbBinB)

Now that we have a prior distribution for each handicap, we just need the denominator of our equation, which is the probability of a given score generally occurring. Based on our model and assumptions, the probability of a given score is the sum of the probability of that score for each and every different handicap. This calculation will be based on the **handicap probability distribution** for each player and the **handicap index** of the hole. However, in order to find a probability for each handicap, we split each previous handicap band into five, so we have an associated probability for each handicap, rather than bands of five. This split is done assuming uniformity across each band. Since five-wide banded data is the most granular available, we make the assumption of intra-band uniformity.

In [ ]:
hcProbMaleGranulated = pd.read_csv('Handicap_Probabilities_Male_Granulated.csv')
hcProbFemaleGranulated = pd.read_csv('Handicap_Probabilities_Female_Granulated.csv')

print('Male Granulated Distribution:\n', hcProbMaleGranulated.shape, '\n', hcProbMaleGranulated.head())
print('Female Granulated Distribution:\n', hcProbFemaleGranulated.shape, '\n', hcProbFemaleGranulated.head())

To find the probability of a score generally occuring, we need to make use of the **law of total probability** and sum over every conditional probability. That is, for the example of a birdie, we need to find the chance that a plus-handicapper makes a birdie, plus a 0-handicapper, plus a 1-handicapper, ..., plus a 53-handicapper, plus a 54-handicapper. This sum will give us the denominator.

In [ ]:
def totalProb(scoreType, holeHandicap, gender):
    """
    Finds the total probability of a given score on a given hole for a given gendered player
    
    Parameters:
        scoreType (string): type of score achieved on a hole; ex: Birdie, Par, etc.
        holeHandicap (int): handicap index of the hole at play; ex: 1, 2, 3, etc.
        gender (string): gender of golfer; ex: male or female
    Returns:
        int: corresponding row in the DataFrame
    """
    
    if scoreType == "Eagle":
        n = 0
    elif scoreType == "Birdie":
        n = 1
    elif scoreType == "Par":
        n = 2
    elif scoreType == "Bogey":
        n = 3
    elif scoreType == "Double Bogey+":
        n = 4
        
    data = [["Eagle", eagleProb], ["Birdie", birdieProb], ["Par", parProb], ["Bogey", bogeyProb], ["Double Bogey+", doublePlusProb]]
    df = pd.DataFrame(data, columns=['Score Type', 'Probability'])
    
    if gender == "M":
        granulatedProb = hcProbMaleGranulated
    if gender == "F":
        granulatedProb = hcProbFemaleGranulated
    
    total = 0
    for x in range(56):
        if x - holeHandicap < 0:
            total += (df.iloc[n]['Probability'] * granulatedProb.iloc[x]['Percentage of Golfers'])
        elif x - holeHandicap < 18:
            if n < 1:
                total += 0
            else:
                total += (df.iloc[n-1]['Probability'] * granulatedProb.iloc[x]['Percentage of Golfers'])
        elif x - holeHandicap < 36:
            if n < 2:
                total += 0
            else:
                total += (df.iloc[n-2]['Probability'] * granulatedProb.iloc[x]['Percentage of Golfers'])
        else:
            if n < 3:
                total += 0
            else:
                total += (df.iloc[n-3]['Probability'] * granulatedProb.iloc[x]['Percentage of Golfers'])
    
    return total

Now that we have a function to find total probability, we will use it for each player.

In [ ]:
totalProbA = totalProb(catScoreA, holeIndex, genderA)
totalProbB = totalProb(catScoreB, holeIndex, genderB)
print('Total probability of a', catScoreA, ':', totalProbA)
print('Total probability of a', catScoreB, ':', totalProbB)

Now that we have our final variable, we can perform our bayesian update.

In [ ]:
probHandicapGivenScoreA = probPlayerA * hcProbBinA / totalProbA
print('The probability that player A is a ', initialHandicapA, ' handicap is', probHandicapGivenScoreA)
probHandicapGivenScoreB = probPlayerB * hcProbBinB / totalProbB
print('The probability that player A is a ', initialHandicapB, ' handicap is', probHandicapGivenScoreB)

Now, we have found the probability of a player being their previously asserted handicap. However, this does not tell us if this is the **most likely** handicap for them. To determine which handicap we should adjust them to, we need to find their probability for *each* handicap. Then, we can find the expected value among all handicaps. The expected value will be their newly assigned handicap.

In [ ]:
allHCsProbPlayerA = []
allHCsCatPlayerA = []

allHCsProbPlayerB = []
allHCsCatPlayerB = []
for x in range(-1,55):
    if (x - holeIndex) >= 36:
        adjCatScoreA = convertScore(scoreA - 3, holePar)
    elif (x - holeIndex) >= 18:
        adjCatScoreA = convertScore(scoreA - 2, holePar)
    elif (x - holeIndex) >= 0:
        adjCatScoreA = convertScore(scoreA - 1, holePar)
    else:
        adjCatScoreA = catScoreA
    
    if (x - holeIndex) >= 36:
        adjCatScoreB = convertScore(scoreB - 3, holePar)
    elif (x - holeIndex) >= 18:
        adjCatScoreB = convertScore(scoreB - 2, holePar)
    elif (x - holeIndex) >= 0:
        adjCatScoreB = convertScore(scoreB - 1, holePar)
    else:
        adjCatScoreB = catScoreB
    
    
    allHCsProbPlayerA.append(d2[adjCatScoreA])
    allHCsCatPlayerA.append(convertScore(scoreA, holePar))
    
    allHCsProbPlayerB.append(d2[adjCatScoreB])
    allHCsCatPlayerB.append(convertScore(scoreB, holePar))

We have now created arrays containg the adjusted scoring probabilities of each player's score for *every* handicap.

Now that we have determined the probability of each score for every handicap, we need the probability of every handicap. As before, this comes from our granulated table of probabilities. Next, the denominator remains as before, our total probabilities

In [ ]:
if genderA == "M":
    granulatedProbA = hcProbMaleGranulated
if genderA == "F":
    granulatedProbA = hcProbFemaleGranulated
if genderA == "M":
    granulatedProbB = hcProbMaleGranulated
if genderA == "F":
    granulatedProbB = hcProbFemaleGranulated

probEachHandicapA = allHCsProbPlayerA * granulatedProbA['Percentage of Golfers'] / totalProbA
probEachHandicapB = allHCsProbPlayerB * granulatedProbB['Percentage of Golfers'] / totalProbB

Now, we have a distribution for the probability that *each* golfer is *each* handicap. We just need to find the expected value from this distribution, the mean. This result will tell us what handicap, based on performance on this hole, each golfer should play as for the next hole.

In [ ]:
sumA = 0
sumB = 0
for x in range(56):
    sumA += x * probEachHandicapA[x]
    sumB += x * probEachHandicapB[x]

print('The updated handicap for Player A is:', sumA)
print('The updated handicap for Player B is:', sumB)

Player A's handicap increases slightly. This makes sense, as Player A recorded a poor score, and, thus, we may believe them to be a slightly worse player than they asserted. However, Player B's new handicap is unexpected. Player B has a good result, however, their handicap also went up. How can this be? Well, Bayesian Updating considers the population distribution as well as the 